Testing of:

Posting and generating Dataclass for IgPostData (posts, albums reels) and IgStData (stories)

Focusing on JK page:
 - posting on my own page
    - photo
    - album
    - normal video
    - reel
 - liking them
 - commenting on them
 - liking the comments
 - posting into the story

Based on third party
 - liking a post/reel
 - commenting a post/reel
 - liking a comment
 - using the third party to publish on JK page

 All the objects created are in the Doc Outputs from the first postings JK
on the JK account link https://docs.google.com/document/d/1SGKB9F9cjemIIsNFRvMl-fdec7CIewiDnGA22NzFZr4/edit

Setting up and imports

In [2]:

"""
main.py:
Schedules and publishes Instagram posts from an Excel spreadsheet,
managing post history and retrying failed uploads.
"""

import os
import json
import random
import time
import datetime
import logging
import pprint


import pandas as pd
from pathlib import Path

from instagrapi.exceptions import ClientError
from instagrapi.types import (
    UserShort,
    StoryMention, 
    StoryMedia, 
    StoryLink,
    StoryHashtag,
    StoryLocation,
    StoryBuild,
    StorySticker,
    StoryStickerLink
)
from instagrapi.story import StoryBuild, StoryBuilder, StoryMention, StorySticker
import instagrapi

from ig_config import Config
from ig_client import IgClient
from ig_data import IgPostData
from ig_utils import IgUtils, create_post_dataframe, save_post_dataframe, correct_orientation
from ig_post_manager import IgPostManager
from ig_st_manager import IgStManager

Opening the Ig session (do not forget to prepare first the config, and the specific session file)
Instantiating the Objects

In [3]:
# Configure logging (optional, but recommended)
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load project configuration file
config_file_path = r"C:\Users\manue\Documents\GitHubMLSN\sn-libraries\notebooks\ig_JK_config.json"  # Replace with the actual path
config = Config(config_file_path=config_file_path)  # Create Config object with path
username = config.get("username")
password = config.get("password")

# Authenticate using IgClient
igcl = IgClient(session_file=config.get("settings_file_path"))  
igcl.login(username, password)

#Instantiate objects
ig_utils = IgUtils(igcl)
ig_post_manager = IgPostManager(igcl)
ig_st_manager = IgStManager(igcl, ig_utils)


INFO:ig_client:Session loaded successfully.
INFO:ig_client:Connected Successfully!


one photo post

In [ ]:
# post test data
# path should be this format from the PC folder
path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\Street Photografy 8 Miraflores & Surquyillo.JPG"

# sometimes the phot is not in the right orientation.
# All need to be passed through this (old photos may not work, but only warnings, no errors)
# No vatiables, just this code (see in album for a set of photos)
correct_orientation(Path(path))

#caption should encapsulate all hashtags with # the mentions with @
caption = "#streetphotography in #Lima #surquillo area by @johnklanik"

# location_place needs to be a Location object.
# Calculated just once, based on the location coming from Instagram
# being the number in https://www.instagram.com/explore/locations/758569403/surquillo/ 
# which needs to be an integer.
# We use the get:location_by_pk from the utils (there are other like get the pk from name)
# and used on all the subsequent uploads if they are from the same location

location: int = 758569403
location_surquillo = ig_utils.get_location_by_pk(location)

In [ ]:
# one photo upload_photo

ig_photo_post = ig_post_manager.upload_photo(
    photo_path=path, caption=caption, location=location_surquillo)

# I removed on ig_post_manager the several attemps
# by changing the MAX_RETRIES = 1  # Maximum number of retries
# as it seems it is working just OK


In [ ]:
#DO NOT NEED TO DO THIS
# this is a direct method to get the whole Media Objetc from the instagrapi
# library if you didn't saved the ig_photo_post

ig_photo_post_pk = igcl.client.media_pk_from_url("https://www.instagram.com/p/C8_19b7o9-g/")
ig_photo_post = igcl.client.media_info(ig_photo_post_pk)

One album of phots and videos

In [ ]:
media_paths = [
    Path(r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\20130816_193318.jpg"),
    Path(r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\papas_video.mp4"),
    Path(r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\20130816_193323.jpg")
    ]      # Add more paths as needed

for path in media_paths:
    correct_orientation(path)  # Correct orientation before uploading
caption = """
“How many #papas are there in Peru?”

"""
# using the same locatn than above

In [ ]:

ig_album_post = ig_post_manager.upload_album(
    paths=media_paths,
    caption=caption,
    location=location_surquillo)


One video_post with upload_video (it is different thatn one reel upload)

In [ ]:
location_larcomar = ig_utils.get_location_by_pk(101440375504236)
path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\Belmond Miraflores Park.mp4"
caption = "Some tim ago I took this where today sits the wonderfull hotel @belmondmiraflorespark"


In [ ]:
ig_video_post = ig_post_manager.upload_video(video_path=path, caption=caption, location=location_surquillo)

One reel post (different from one video post)

In [ ]:
# one reel upload_reel 

path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\prices are great.mp4"
caption = " #vegetables and #fruit prices are great in Peru"
#location same as above



In [ ]:
ig_reel = ig_post_manager.upload_video(path, caption=caption, location= location_surquillo)

In [ ]:
#DO NOT NEED TO DO THIS
# this is a direct method to get the whole Media Objetc from the instagrapi
# library if you didn't saved the ig_photo_post

ig_reel_pk = igcl.client.media_pk_from_url("https://www.instagram.com/p/C8_7O9yMPzi/")
ig_reel = igcl.client.media_info(ig_reel_pk)
print(ig_reel)

Commenting and liking all kinds
 - Not converted to libraries yet
 - using directly the instagrapi methods directly

In [ ]:
# Commenting on the above posts
comment_photo = "great street photography"
comment_album = "they say they have like four thousand varieties"
comment_reel = "and those are soles, not dollars"
media_photo_id: str = "3404677161660178336_67318685241"    # in case you picked somewhere else
media_album_id: str = ig_album_post.media_id # this is the normal to do using the IgPostData dataclass
media_reel_id: str = "3404700356472536290_67318685241"


comment = igcl.client.media_comment(media_photo_id, comment_photo)
time.sleep(63)
comment = igcl.client.media_comment(media_album_id, comment_album)
time.sleep(97)
comment = igcl.client.media_comment(media_reel_id, comment_reel)

Comment on a third party post using the link of the post

In [ ]:
# try to comment on the post by ahiesnada  https://www.instagram.com/p/CnHw_vmrXJz/
media_pk = igcl.client.media_pk_from_url("https://www.instagram.com/p/CnHw_vmrXJz/")
print("media_pk: ",media_pk)

insights_media = igcl.client.insights_media(media_pk)
print(f"insights media\n{insights_media}")

media_id = igcl.client.media_id(media_pk)
print("media_id: ",media_id)

comment_on_ahiesnada = igcl.client.media_comment(media_id, "I love it. This is great #streetphotography")


In [ ]:
print(comment_on_ahiesnada)

Likes on posts, reels and comments

In [ ]:
# Like my reels/posts:
# BECAREFUL NOT TO CALL THIS TWICE

album_like = igcl.client.media_like(ig_album_post.media_id)



In [ ]:
# like third party posts/reels
media_pk = igcl.client.media_pk_from_url("https://www.instagram.com/p/CnXOdOdKUTp/")
media_id: str = igcl.client.media_id(media_pk)
like = igcl.client.media_like(ig_album_post.media_id)

In [ ]:
# like on my comments on my account

comment_pk: int = comment.pk   # normally extract firectly from the Object
like_on_reel = igcl.client.comment_like(comment_pk)

In [ ]:
# like comments I did on other accounts
# extract directly the pk in the call

comment_on_ahiesnada_like = igcl.client.comment_like(comment_on_ahiesnada.pk)

Upload to story

In [10]:
# ONLY THE METHOD TO SHARE A VIDEO OR REEL TO STORY HAS BEEN CREATED
# My own story
# from my reel: https://www.instagram.com/reel/C8_7O9yMPzi/
# from photo https://www.instagram.com/p/C85AI2ps9NR/
# from my album: https://www.instagram.com/p/C8_6ramNXoX/?utm_source=ig_web_button_share_sheet&igsh=ZDNlZDc0MzIxNw%3D%3D&img_index=3
# from a third party media https://www.instagram.com/reel/C9AMD8HonUx/

#  IT STILL NOT WELL PROOFED.

# THE METHOD WORKS WELL AS IN THE EXAMPLE BELOW,
# BUT WE ARE LACKING THE UPLOAD OF A PHOTO,
# THE CONVERSION OF A PHOTO INTO VIDEO AND UPLOAD IT
# AND THE UPLOAD OF A REEL WITHOUT SO MUCH INFO

# ALSO INSTAGRAPI DOES NOT WORKS WELL AND IT ONLY UPLOADS THE REEL, BUT NOT THE OTHER INFO

other story check 
example = cl.user_info_by_username('example')

In [ ]:
# story reel from other source: "https://www.instagram.com/p/C8-jIjnI3z8/"
# location Id = plaza de armas de Lima = 1380181525548680

# REMEMBER the Story is using the UserShort object, not the user object

third_party = "https://www.instagram.com/reel/C9AMD8HonUx/"   
st_media_pk = igcl.client.media_pk_from_url(third_party)
st_media_path = igcl.client.video_download(st_media_pk)
st_source_long = igcl.client.user_info_by_username("europexplores")
st_source = UserShort(
    pk=st_source_long.pk,
    username=st_source_long.username,
    full_name=st_source_long.full_name,
    profile_pic_url=st_source_long.profile_pic_url
)
st_caption = " agree with @europexplores"
st_location = ig_utils.get_location_by_pk(1380181525548680)
st_hashtag = igcl.client.hashtag_info("fiestaspatrias")
# no stickers this time

story_video = igcl.client.video_upload_to_story(
    st_media_path,
    caption=st_caption,
    mentions=[StoryMention(user=st_source, x= 0.5, y=0.75, width=0.45, height=0.1)],
    locations=[StoryLocation(location=st_location)],
    links=[StoryLink(webUri="https://www.youtube.com/watch?v=y9ezAdXMUrw")],
    hashtags=[StoryHashtag(hashtag=st_hashtag, x=0.23, y=0.32, width=0.3, height=.18)],
    medias = [StoryMedia(media_pk=st_media_pk, x=0.5, y=0.6, width=0.4, height=0.6)]
    )

In [ ]:
print(f"st_reel_url ({type(st_reel_url).__name__}): {st_reel_url}")
print("\n")  # Empty print for a blank line
print(f"st_media_pk ({type(st_media_pk).__name__}): {st_media_pk}")
print("\n")
print(f"st_media_path ({type(st_media_path).__name__}): {st_media_path}")
print("\n")
print(f"st_source ({type(st_source).__name__}): {st_source}")
print("\n")
print(f"st_caption ({type(st_caption).__name__}): {st_caption}")
print("\n")
print(f"st_location ({type(st_location).__name__}): {st_location}")
print("\n")
print(f"st_hashtag ({type(st_hashtag).__name__}): {st_hashtag}")

